In [1]:
# Project Description: Creating Line Plots of GDP Data

In [2]:
import csv
import pygal

In [3]:
def read_csv_as_nested_dict(filename, keyfield, separator, quote):
    """
    Inputs:
      filename  - Name of CSV file
      keyfield  - Field to use as key for rows
      separator - Character that separates fields
      quote     - Character used to optionally quote fields

    Output:
      Returns a dictionary of dictionaries where the outer dictionary
      maps the value in the key_field to the corresponding row in the
      CSV file.  The inner dictionaries map the field names to the
      field values for that row.
    """
    result = {}
    with open(filename, mode='r', newline='') as csvfile:
        reader = csv.DictReader(csvfile, delimiter=separator, quotechar=quote)
        for row in reader:
            key = row[keyfield]
            result[key] = row
    return result

In [4]:
# Example usage:
# data = read_csv_as_nested_dict('isp_gdp.csv', 'Country Name', ',', '"')
# print(data)

{'Aruba': {'Country Name': 'Aruba', 'Country Code': 'ABW', 'Indicator Name': 'GDP (current US$)', 'Indicator Code': 'NY.GDP.MKTP.CD', '1960': '', '1961': '', '1962': '', '1963': '', '1964': '', '1965': '', '1966': '', '1967': '', '1968': '', '1969': '', '1970': '', '1971': '', '1972': '', '1973': '', '1974': '', '1975': '', '1976': '', '1977': '', '1978': '', '1979': '', '1980': '', '1981': '', '1982': '', '1983': '', '1984': '', '1985': '', '1986': '', '1987': '', '1988': '', '1989': '', '1990': '', '1991': '', '1992': '', '1993': '', '1994': '1330167598', '1995': '1320670391', '1996': '1379888268', '1997': '1531843575', '1998': '1665363128', '1999': '1722798883', '2000': '1873452514', '2001': '1920262570', '2002': '1941094972', '2003': '2021301676', '2004': '2228279330', '2005': '2331005587', '2006': '2421474860', '2007': '2623726257', '2008': '2791960894', '2009': '2498932961', '2010': '2467703911', '2011': '2584463687', '2012': '', '2013': '', '2014': '', '2015': '', '2016': ''}, '

In [5]:
def build_plot_values(gdpinfo, gdpdata):
    """
    Inputs:
      gdpinfo - GDP data information dictionary
      gdpdata - A single country's GDP stored in a dictionary whose
                keys are strings indicating a year and whose values
                are strings indicating the country's corresponding GDP
                for that year.

    Output: 
      Returns a list of tuples of the form (year, GDP) for the years
      between "min_year" and "max_year", inclusive, from gdpinfo that
      exist in gdpdata.  The year will be an integer and the GDP will
      be a float.
    """
    min_year = int(gdpinfo['min_year'])
    max_year = int(gdpinfo['max_year'])
    
    plot_values = []
    for year in range(min_year, max_year + 1):
        year_str = str(year)
        if year_str in gdpdata and gdpdata[year_str]:
            try:
                gdp = float(gdpdata[year_str])
                plot_values.append((year, gdp))
            except ValueError:
                continue  # Skip if the GDP value is not a valid float
    
    return plot_values

In [6]:
# Example usage:
# gdpinfo = {
#     'min_year': '1960',
#     'max_year': '2016',
#     'country_name': 'Country Name',
#     'country_code': 'Country Code',
#     'indicator_name': 'Indicator Name',
#     'indicator_code': 'Indicator Code'
# }
# gdpdata = {
#     '1960': '1000',
#     '1961': '1100',
#     '1962': '1200',
#     '2015': '1300',
#     '2016': '1400'
# }
# plot_values = build_plot_values(gdpinfo, gdpdata)
# print(plot_values)

In [7]:
def build_plot_dict(gdpinfo, country_list):
    """
    Inputs:
      gdpinfo      - GDP data information dictionary
      country_list - List of strings that are country names

    Output:
      Returns a dictionary whose keys are the country names in
      country_list and whose values are lists of XY plot values 
      computed from the CSV file described by gdpinfo.

      Countries from country_list that do not appear in the
      CSV file should still be in the output dictionary, but
      with an empty XY plot value list.
    """
    # Read the CSV file into a nested dictionary
    gdp_data = read_csv_as_nested_dict(gdpinfo['gdpfile'], 
                                       gdpinfo['country_name'], 
                                       gdpinfo['separator'], 
                                       gdpinfo['quote'])

    plot_dict = {}
    for country in country_list:
        if country in gdp_data:
            plot_dict[country] = build_plot_values(gdpinfo, gdp_data[country])
        else:
            plot_dict[country] = []

    return plot_dict

In [8]:
# Example usage:
# gdpinfo = {
#     'filename': 'isp_gdp.csv',
#     'min_year': '1960',
#     'max_year': '2016',
#     'country_name': 'Country Name',
#     'country_code': 'Country Code',
#     'indicator_name': 'Indicator Name',
#     'indicator_code': 'Indicator Code',
#     'separator': ',',
#     'quote': '"'
# }
# country_list = ['Aruba', 'Andorra', 'Afghanistan', 'Angola']
# plot_dict = build_plot_dict(gdpinfo, country_list)
# print(plot_dict)

In [9]:
def render_xy_plot(gdpinfo, country_list, plot_file):
    """
    Inputs:
      gdpinfo      - GDP data information dictionary
      country_list - List of strings that are country names
      plot_file    - String that is the output plot file name

    Output:
      Returns None.

    Action:
      Creates an SVG image of an XY plot for the GDP data
      specified by gdpinfo for the countries in country_list.
      The image will be stored in a file named by plot_file.
    """
    # Build the plot dictionary using previously defined function
    plot_dict = build_plot_dict(gdpinfo, country_list)
    
    # Create a XY chart using Pygal
    xy_chart = pygal.XY(stroke=False)
    xy_chart.title = 'GDP data from World Bank'
    xy_chart.x_title = 'Year'
    xy_chart.y_title = 'GDP in current US dollars'
    
    # Add data to the chart
    for country, plot_values in plot_dict.items():
        xy_chart.add(country, plot_values)
    
    # Render the chart to an SVG file
    xy_chart.render_to_file(plot_file)

In [10]:
# Example usage:
# gdpinfo = {
#     'filename': 'isp_gdp.csv',
#     'min_year': '1960',
#     'max_year': '2016',
#     'country_name': 'Country Name',
#     'country_code': 'Country Code',
#     'indicator_name': 'Indicator Name',
#     'indicator_code': 'Indicator Code',
#     'separator': ',',
#     'quote': '"'
# }
# country_list = ['Aruba', 'Andorra', 'Afghanistan', 'Angola']
# plot_file = 'gdp_plot.svg'

In [11]:
def test_render_xy_plot():
    """
    Code to exercise render_xy_plot and generate plots from
    actual GDP data.
    """
    gdpinfo = {
        "gdpfile": "isp_gdp.csv",
        "separator": ",",
        "quote": '"',
        "min_year": 1960,
        "max_year": 2015,
        "country_name": "Country Name",
        "country_code": "Country Code"
    }

    render_xy_plot(gdpinfo, [], "isp_gdp_xy_none.svg")
    render_xy_plot(gdpinfo, ["China"], "isp_gdp_xy_china.svg")
    render_xy_plot(gdpinfo, ["United Kingdom", "United States"],
                   "isp_gdp_xy_uk+usa.svg")


In [12]:
test_render_xy_plot()